In [27]:
%reload_ext autoreload
%autoreload 2

In [28]:
import numpy as np
import similarity

In [30]:
def my_metric(x, y, param=None):
    if param is not None:
        return param
    return x.reshape(-1) @ y.reshape(-1) / (np.linalg.norm(x) * np.linalg.norm(y))

# similarity.register("measure.my_metric")({
#     "_target_": "similarity.Metric",
#     "metric": None,
#     "fit_score": {
#         "_target_": "config_utils.DictModule",
#         "module": my_metric,
#         "in_keys": [["X", "x"], ["Y", "y"]],
#         "out_keys": ["score"]
#     }
# })

# similarity.register("measure.my_metric.fit_score")({
#     "_target_": "config_utils.DictModule",
#     "module": my_metric,
#     "in_keys": [["X", "x"], ["Y", "y"]],
#     "out_keys": ["score"]
# })
similarity.register(my_metric, "measure.my_metric.fit_score")

print(similarity.make("measure.my_metric.fit_score"))
metric = similarity.make("measure.my_metric")
print(metric)

X, Y = np.random.randn(100, 30), np.random.randn(100, 30)
print(metric.fit_score(X, Y))

OrderedDict([('x', <Parameter "x">), ('y', <Parameter "y">), ('param', <Parameter "param=None">)])


AssertionError: 

In [36]:
similarity.register(my_metric, "measure.test")
test_metric = similarity.make("measure.test")
print(test_metric)
print(test_metric(X=X, Y=Y))
# TODO: allow args
print(test_metric(X, X=Y, Y=Y))

OrderedDict([('x', <Parameter "x">), ('y', <Parameter "y">), ('param', <Parameter "param=None">)])
{'x': 1}
DictModule(
  module=<function my_metric at 0x000001C2381B6430>
  file="C:\Users\Nathan Cloos\AppData\Local\Temp\ipykernel_11024\75367330.py"
  in_keys=[['X', 'x'], ['Y', 'y']]
  out_keys=['score']
)
{'score': 0.02129918535696253}
{'score': 1.0}


In [42]:
# simply register as a function
similarity.register(my_metric, "measure.my_metric-param10", param=10)
metric = similarity.make("measure.my_metric-param10")
metric(X, Y)

# register measure as a class where fit_score method (useful to have a common interface)
similarity.register(my_metric, "measure.my_metric-param20.fit_score", param=20)
metric = similarity.make("measure.my_metric-param20")
metric.fit_score(X, Y)

{'score': 20}

In [64]:
# register a class
# @similarity.register(
#     "measure.my_metric2",
#     interface={
#         "fit_score": "__call__"
#     },
#     alpha=1.0
# )
class MyMetric2:
    def __init__(self, alpha):
        self.alpha = alpha  # TODO

    # @similarity.register("measure.my_metric2.fit")
    def fit(self, x, y):
        pass

    # @similarity.register("measure.my_metric2.fit_score")
    def fit_score(self, x, y):
        # return x.reshape(-1) @ y.reshape(-1)
        return self.alpha
    

similarity.register(MyMetric2, "measure.my_metric2-alpha1", alpha=1)
similarity.register(MyMetric2, "measure.my_metric2-alpha5e-1", alpha=0.5)
similarity.register(MyMetric2, "measure.my_metric2-alpha0", alpha=0)

['self', 'x', 'y']
['self', 'x', 'y']
['self', 'x', 'y']


In [65]:
metric = similarity.make("measure.my_metric2-alpha5e-1")
print(metric._fit_score)
print(metric.fit_score(X, Y))

DictModule(
  module=<function MyMetric2.fit_score at 0x000001C238423940>
  file="C:\Users\Nathan Cloos\AppData\Local\Temp\ipykernel_11024\3270678386.py"
  in_keys=[['metric', 'self'], ['X', 'x'], ['Y', 'y']]
  out_keys=['score']
)
{'score': 0.5}


In [53]:
# TODO: doesn't make sense to fit without storing state in "self"
def fit_my_metric(x, y):
    pass

def fit_score_my_metric(x, y):
    return x.reshape(-1) @ y.reshape(-1)

similarity.register(fit_my_metric, "measure.my_metric3.fit")
similarity.register(fit_score_my_metric, "measure.my_metric3.fit_score")
metric3 = similarity.make("measure.my_metric3")
print(metric3._fit)
print(metric3._fit_score)
print(metric3.fit_score(X, Y))

DictModule(
  module=<function fit_my_metric at 0x000001C23842F790>
  file="C:\Users\Nathan Cloos\AppData\Local\Temp\ipykernel_11024\2271397120.py"
  in_keys=[['X', 'x'], ['Y', 'y']]
  out_keys=['score']
)
DictModule(
  module=<function fit_score_my_metric at 0x000001C2384CC670>
  file="C:\Users\Nathan Cloos\AppData\Local\Temp\ipykernel_11024\2271397120.py"
  in_keys=[['X', 'x'], ['Y', 'y']]
  out_keys=['score']
)
{'score': 63.40865252067996}


In [18]:
print(similarity.registry)

defaultdict(<function <lambda> at 0x000001C2380A89D0>, {'measure': defaultdict(<function <lambda> at 0x000001C2380CB3A0>, {'my_metric': {'_target_': 'similarity.Metric', 'metric': None, 'fit_score': DictModule(
  module=<function my_metric at 0x000001C2380FA4C0>
  file="C:\Users\Nathan Cloos\AppData\Local\Temp\ipykernel_11024\3656328089.py"
  in_keys=[['X', 'x'], ['Y', 'y']]
  out_keys=['score']
), 'fit': None, 'score': None}, 'test': DictModule(
  module=<function my_metric at 0x000001C2380FA4C0>
  file="C:\Users\Nathan Cloos\AppData\Local\Temp\ipykernel_11024\3656328089.py"
  in_keys=[['X', 'x'], ['Y', 'y']]
  out_keys=['score']
)})})


In [11]:
metric = similarity.make("measure.my_metric")
print(metric)

{'_target_': 'similarity.Metric', 'metric': None, 'fit_score': DictModule(
  module=<function my_metric at 0x000001C23807FB80>
  file="C:\Users\Nathan Cloos\AppData\Local\Temp\ipykernel_11024\2437551110.py"
  in_keys=[['X', 'x'], ['Y', 'y']]
  out_keys=['score']
), 'fit': None, 'score': None}
Metric[fit(X, Y), score(X, Y), fit_score(X, Y)]


In [12]:
print(metric._fit_score)

DictModule(
  module=<function my_metric at 0x000001C23807FB80>
  file="C:\Users\Nathan Cloos\AppData\Local\Temp\ipykernel_11024\2437551110.py"
  in_keys=[['X', 'x'], ['Y', 'y']]
  out_keys=['score']
)


In [13]:
X, Y = np.random.randn(100, 30), np.random.randn(100, 30)
print(metric.fit_score(X, Y))


{'score': -36.80423758361501}
